In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!nvidia-smi --query-gpu=gpu_name,driver_version,memory.total --format=csv

In [ ]:
pip install gdown

In [ ]:
import gdown

# pretrained model

#url = 'https://drive.google.com/uc?id=1xn_lWDN8w9KfjRpzMDvlpka7USUIdwb-'
#output = 'wlasl-100-ntu60.pth'
url = 'https://drive.google.com/uc?id=1TCW6pyhSHiDdmwwOacf1fUG_8jXLuFcM' # epoch 9 (83% ish) on autsl
output = 'epoch-9.pth'
gdown.download(url, output, quiet=False)

In [ ]:
# train set 

#url = 'https://drive.google.com/uc?id=1-3qjOT34fIBIvumj805Wb4ciS4CoohxY'
url = 'https://drive.google.com/uc?id=1--0viaiuY-xeSFpr77FOmu8cxuIVPpJI'
output = 'train-phoenix2014t.pkl'
gdown.download(url, output, quiet=False)

In [ ]:
# valid set

#url = 'https://drive.google.com/uc?id=1-EqyTWil4acQS1S_fYWj-33lG7AJR0r5'
url = 'https://drive.google.com/uc?id=1--jabpE5tu8IkUCOcj3prfkkJFlWe_dV'
output = 'valid-phoenix2014t.pkl'
gdown.download(url, output, quiet=False)

**install mmaction2**

In [ ]:
# Check nvcc version
!nvcc -V
# Check GCC version
!gcc --version

In [ ]:
# install dependencies: (use cu101 because colab has CUDA 10.1)
!pip install -U torch==1.8.0+cu101 torchvision==0.9.0+cu101 torchtext==0.9.0 torchaudio==0.8.0 -f https://download.pytorch.org/whl/torch_stable.html

# install mmcv-full thus we could use CUDA operators
!pip install mmcv-full -f https://download.openmmlab.com/mmcv/dist/cu101/torch1.8.0/index.html

# Install mmaction2
!rm -rf mmaction2
!git clone https://github.com/open-mmlab/mmaction2.git
%cd mmaction2

!pip install -e .

# Install some optional requirements
!pip install -r requirements/optional.txt

In [ ]:
# Check Pytorch installation
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

# Check MMAction2 installation
import mmaction
print(mmaction.__version__)

# Check MMCV installation
from mmcv.ops import get_compiling_cuda_version, get_compiler_version
print(get_compiling_cuda_version())
print(get_compiler_version())

**restart, then, train**

In [ ]:
#!mkdir results

In [ ]:
import pickle
with open('/kaggle/working/train-phoenix2014t.pkl','rb') as f:
  data = pickle.load(f)
len(data)

In [ ]:
train = data[100000:106536]
print(len(train))

In [ ]:
with open('/kaggle/working/train2.pkl', 'wb') as f:
  pickle.dump(train, f)

In [ ]:
%%writefile /kaggle/working/mmaction2/configs/skeleton/posec3d/slowonly_r50_u48_240e_ntu60_xsub_keypoint.py

model = dict(
    type='Recognizer3D',
    backbone=dict(
        type='ResNet3dSlowOnly',
        depth=50,
        pretrained=None,
        in_channels=17,
        base_channels=32,
        num_stages=3,
        out_indices=(2, ),
        stage_blocks=(4, 6, 3),
        conv1_stride_s=1,
        pool1_stride_s=1,
        inflate=(0, 1, 1),
        spatial_strides=(2, 2, 2),
        temporal_strides=(1, 1, 2),
        dilations=(1, 1, 1)),
    cls_head=dict(
        type='I3DHead',
        in_channels=512,
        num_classes=226,
        spatial_type='avg',
        dropout_ratio=0.5),
    train_cfg=dict(),
    test_cfg=dict(feature_extraction=True))

dataset_type = 'PoseDataset'
ann_file_train = '/kaggle/working/train-phoenix2014t.pkl'
ann_file_val = '/kaggle/working/train2.pkl'
left_kp = [1, 3, 5, 6, 7, 8, 9, 10]
right_kp = [2, 4, 11, 12, 13, 14, 15, 16]
train_pipeline = [
    dict(type='UniformSampleFrames', clip_len=48),
    dict(type='PoseDecode'),
    dict(type='PoseCompact', hw_ratio=1., allow_imgpad=True),
    dict(type='Resize', scale=(-1, 64)),
    dict(type='RandomResizedCrop', area_range=(0.56, 1.0)),
    dict(type='Resize', scale=(56, 56), keep_ratio=False),
    dict(type='Flip', flip_ratio=0.5, left_kp=left_kp, right_kp=right_kp),
    dict(
        type='GeneratePoseTarget',
        sigma=0.6,
        use_score=True,
        with_kp=True,
        with_limb=False),
    dict(type='FormatShape', input_format='NCTHW'),
    dict(type='Collect', keys=['imgs', 'label'], meta_keys=[]),
    dict(type='ToTensor', keys=['imgs', 'label'])
]
val_pipeline = [
    dict(type='UniformSampleFrames', clip_len=48, num_clips=1, test_mode=True),
    dict(type='PoseDecode'),
    dict(type='PoseCompact', hw_ratio=1., allow_imgpad=True),
    dict(type='Resize', scale=(-1, 64)),
    dict(type='CenterCrop', crop_size=64),
    dict(
        type='GeneratePoseTarget',
        sigma=0.6,
        use_score=True,
        with_kp=True,
        with_limb=False),
    dict(type='FormatShape', input_format='NCTHW'),
    dict(type='Collect', keys=['imgs', 'label'], meta_keys=[]),
    dict(type='ToTensor', keys=['imgs'])
]
test_pipeline = [
    dict(
        type='UniformSampleFrames', clip_len=48, num_clips=10, test_mode=True),
    dict(type='PoseDecode'),
    dict(type='PoseCompact', hw_ratio=1., allow_imgpad=True),
    dict(type='Resize', scale=(-1, 64)),
    dict(type='CenterCrop', crop_size=64),
    dict(
        type='GeneratePoseTarget',
        sigma=0.6,
        use_score=True,
        with_kp=True,
        with_limb=False,
        double=True,
        left_kp=left_kp,
        right_kp=right_kp),
    dict(type='FormatShape', input_format='NCTHW'),
    dict(type='Collect', keys=['imgs', 'label'], meta_keys=[]),
    dict(type='ToTensor', keys=['imgs'])
]
data = dict(
    videos_per_gpu=20,
    workers_per_gpu=2,
    test_dataloader=dict(videos_per_gpu=1),
    train=dict(
        type=dataset_type,
        ann_file=ann_file_train,
        data_prefix='',
        pipeline=train_pipeline),
    val=dict(
        type=dataset_type,
        ann_file=ann_file_val,
        data_prefix='',
        pipeline=val_pipeline),
    test=dict(
        type=dataset_type,
        ann_file=ann_file_val,
        data_prefix='',
        pipeline=test_pipeline))
# optimizer
optimizer = dict(
    type='SGD', lr=0.01, momentum=0.9,
    weight_decay=0.0003)  # this lr is used for 8 gpus
optimizer_config = dict(grad_clip=dict(max_norm=40, norm_type=2))
# learning policy
lr_config = dict(policy='CosineAnnealing', by_epoch=False, min_lr=0)
total_epochs = 120
checkpoint_config = dict(interval=1)
workflow = [('train', 1)]
evaluation = dict(
    interval=1,
    metrics=['top_k_accuracy', 'mean_class_accuracy'],
    topk=(1, 5))
log_config = dict(
    interval=250, hooks=[
        dict(type='TextLoggerHook'),
    ])
dist_params = dict(backend='nccl')
log_level = 'INFO'
work_dir = '/kaggle/working/results/'
load_from = None
resume_from = None
find_unused_parameters = False

In [ ]:
!python /kaggle/working/mmaction2/tools/test.py /kaggle/working/mmaction2/configs/skeleton/posec3d/slowonly_r50_u48_240e_ntu60_xsub_keypoint.py \
    /kaggle/working/epoch-9.pth \
    --out /kaggle/working/train-result-raw-100k-106k.pkl

In [ ]:
!python /kaggle/working/mmaction2/tools/train.py /kaggle/working/mmaction2/configs/skeleton/posec3d/slowonly_r50_u48_240e_ntu60_xsub_keypoint.py \
    --validate --seed 0 --deterministic --gpu-ids 0

In [ ]:
!zip -r -q epoch-13.zip /kaggle/working/results/

In [ ]:
from IPython.display import FileLink
FileLink(r'valid-result-raw.pkl')